In [614]:
from tqdm import tqdm
import pandas as pd
from binance.client import Client

import numpy as np
import ta
from ta.momentum import RSIIndicator, StochRSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange 
import re as re
from sqlalchemy.sql import exists  
from sqlalchemy import select
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker, relationship

In [615]:
client = Client()

In [616]:
from sqlalchemy import create_engine

pd.set_option("display.max_rows", 100, "display.max_columns", 100)

In [617]:
coins = ('BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','LUNAUSDT',
  'DOGEUSDT','AVAXUSDT','SHIBUSDT','MATICUSDT','LTCUSDT','UNIUSDT','ALGOUSDT','TRXUSDT',
         'LINKUSDT','MANAUSDT','ATOMUSDT','VETUSDT')

In [618]:
fast = 7
slow = 25


In [619]:
def getminutedata(symbol, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'1m',lookback + ' days ago UTC' ))

    frame = frame.iloc[:,:6]
    frame.drop_duplicates()
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms')
    return frame

In [620]:
engine = db.create_engine('sqlite:///75days1901.db')
#echo = True

In [621]:
Session = sessionmaker(bind=engine)
session = Session()
#users = session.query(Time)
#exists = db.session.query(User.id).filter_by(name='davidism').first() is not None

In [623]:
#

In [624]:
#for coin in tqdm(coins):
   #getminutedata(coin,'20').to_sql(coin, engine,if_exists="append", index=False)


In [625]:
test=pd.read_sql('BTCUSDT',engine).set_index('Time')

In [626]:
def groups_from_conditions(cond1, cond2):
    '''
    assign a unique non-NaN integer to each group as defined by the rules
    '''
    n = len(cond1)
    
    group_idx = -1
    groups = np.zeros(n)

    curr_state = 0    # 0 = not in a group, 1 = in a group
    for n in range(n):
        if curr_state == 0:
            # Currently not in a group
            if cond1[n] == 1:
                # Detected start of a group. so:
                # switch the state to 1 ie in a group
                curr_state = 1
                # get a new group_idx
                group_idx = group_idx + 1
                # assign it to the output for element n
                groups[n] = group_idx
            else:
                # no start of the group detected, we are not in a group so mark as NaN
                groups[n] = np.NaN

        else: 
            # current_state == 1 so we are in a group
            if cond2[n] == 1:
                # detected end of group -- switch state to 0
                curr_state = 0
            # as we are in a group assign current group_idx. Note that this happens for the element
            # for which cond2[n] == 1 as well, ie this element is included
            groups[n] = group_idx

    return groups

In [627]:
def smas(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['SMA_fast'] = np.log(df.Close.rolling(7).mean())
    df['SMA_slow'] = np.log(df.Close.rolling(25).mean())
    df['buy'] = np.where(df['SMA_fast'] > df['SMA_slow'], 1 , 0)
    df['sell'] = np.where(df['SMA_fast'] < df['SMA_slow'], 1 , 0)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.dropna(inplace=True)
    return df



In [628]:
smas(test)

,Open,High,Low,Close,Volume,return,SMA_fast,SMA_slow,buy,sell,groups,position,strategyreturn
Time,,,,,,,,,,,,,
2021-12-30 15:20:00,47489.67,47489.68,47475.61,47475.62,8.76160,0.001408,10.765663,10.765507,1,0,0.0,1,0.000000
2021-12-30 15:25:00,47383.89,47394.33,47366.47,47387.05,10.26947,-0.001867,10.766115,10.765454,1,0,0.0,1,-0.001867
2021-12-30 15:30:00,47356.85,47357.16,47315.42,47315.42,12.00536,-0.001513,10.766135,10.765325,1,0,0.0,1,-0.001513
2021-12-30 15:35:00,47334.11,47334.12,47316.16,47320.49,9.36621,0.000107,10.765945,10.765196,1,0,0.0,1,0.000107
2021-12-30 15:40:00,47319.81,47369.69,47301.66,47362.78,17.43743,0.000893,10.766004,10.765099,1,0,0.0,1,0.000893
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19 12:45:00,41990.01,42010.00,41893.62,42001.00,100.57046,0.000272,10.646784,10.643508,1,0,144.0,1,0.000272
2022-01-19 12:50:00,42096.00,42105.11,42080.88,42082.87,11.13828,0.001947,10.646895,10.644204,1,0,144.0,1,0.001947
2022-01-19 12:55:00,42132.76,42147.30,42128.78,42128.79,10.66275,0.001091,10.647182,10.644860,1,0,144.0,1,0.001091


In [629]:
def bb(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    bb_indicator = BollingerBands(df['Close'])
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['bb_high'] = bb_indicator.bollinger_hband()
    df['bb_low'] = bb_indicator.bollinger_lband()
    df['moving_avg'] = bb_indicator.bollinger_mavg()
    df['buy'] = np.where(df['Close'] * 0.8 < df['bb_low'], 1, 0)
    df['sell'] = np.where(df['Close'] > df['bb_high'], 1, 0)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.to_string("x")
    df.dropna(inplace=True)
    return df

In [630]:
def stoch(df):
    # default = 14, 3, 3, window, smooth1, smooth2)

    #df['stochrsi_d'] = stochrsi.stochrsi_d()
    #df['stochrsi_k'] = stochrsi.stochrsi_k()
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    stochrsi = StochRSIIndicator(df['Close'], 14, 3 ,3)
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['stochrsi'] = stochrsi.stochrsi()
    df['buy'] = np.where(df['stochrsi'] > 0.80, 1, 0)
    df['sell'] = np.where(df['stochrsi'] < 0.20, 1, 0)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.to_string("x")
    df.dropna(inplace=True)
    return df

In [631]:
smas(test)

,Open,High,Low,Close,Volume,return,SMA_fast,SMA_slow,buy,sell,groups,position,strategyreturn
Time,,,,,,,,,,,,,
2021-12-30 15:20:00,47489.67,47489.68,47475.61,47475.62,8.76160,0.001408,10.765663,10.765507,1,0,0.0,1,0.000000
2021-12-30 15:25:00,47383.89,47394.33,47366.47,47387.05,10.26947,-0.001867,10.766115,10.765454,1,0,0.0,1,-0.001867
2021-12-30 15:30:00,47356.85,47357.16,47315.42,47315.42,12.00536,-0.001513,10.766135,10.765325,1,0,0.0,1,-0.001513
2021-12-30 15:35:00,47334.11,47334.12,47316.16,47320.49,9.36621,0.000107,10.765945,10.765196,1,0,0.0,1,0.000107
2021-12-30 15:40:00,47319.81,47369.69,47301.66,47362.78,17.43743,0.000893,10.766004,10.765099,1,0,0.0,1,0.000893
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19 12:45:00,41990.01,42010.00,41893.62,42001.00,100.57046,0.000272,10.646784,10.643508,1,0,144.0,1,0.000272
2022-01-19 12:50:00,42096.00,42105.11,42080.88,42082.87,11.13828,0.001947,10.646895,10.644204,1,0,144.0,1,0.001947
2022-01-19 12:55:00,42132.76,42147.30,42128.78,42128.79,10.66275,0.001091,10.647182,10.644860,1,0,144.0,1,0.001091


In [632]:
def rsistrat(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    rsi_indicator = RSIIndicator(df['Close'])
    #df = df.iloc[16: , :]
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['rsi'] = rsi_indicator.rsi()
    df['buy'] = np.where(df['rsi'] < 30, 1, 0)
    df['sell'] = np.where(df['rsi'] > 70, 1, 0)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.to_string('rsi')
    df.dropna(inplace=True)
    return df

In [633]:
rsistrat(test).to_string('testsss')

In [634]:
def compare(df, func, func2):
    df = df.copy()
    df = df.resample('5min').ffill()
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['rsi'] = func2(df)['rsi']
    df['func1buy'] = func(df)['buy']
    df['func2buy'] = func2(df)['buy']
    df['func1sell'] = func(df)['sell']
    df['func2sell'] = func2(df)['sell']
    df['overlap-buy'] = (df['func1buy'] == 1) & (df['func2buy'] ==1)
    df['overlap-sell'] = (df['func1sell'] == 1) & (df['func2sell'] ==1)
    test_cond1 = df['overlap-buy'].values
    test_cond2 = df['overlap-sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
  
    #df['func1buy'].to_string('buy')
    df.dropna(inplace=True)
    df.to_string('overlap')
    return  df
    

In [635]:
print(compare(test,bb,rsistrat))

                         Open      High       Low     Close    Volume  \
Time                                                                    
2021-12-31 07:35:00  47032.87  47036.01  46968.50  46995.64  51.42702   
2021-12-31 07:40:00  46964.26  46964.64  46904.91  46931.31  39.28713   
2021-12-31 07:45:00  46877.89  46916.69  46874.51  46890.02  39.84190   
2021-12-31 07:50:00  46858.79  46860.43  46836.86  46836.87  41.17821   
2021-12-31 07:55:00  47011.49  47018.42  46982.97  46999.87  27.35553   
...                       ...       ...       ...       ...       ...   
2022-01-18 20:55:00  41626.26  41655.08  41594.43  41645.59   9.39491   
2022-01-18 21:00:00  41784.97  41822.06  41730.52  41732.01  32.34833   
2022-01-18 21:05:00  41749.98  41756.03  41740.06  41750.70   6.67478   
2022-01-18 21:10:00  41834.99  41896.51  41828.17  41893.26  56.77119   
2022-01-18 21:15:00  41960.94  42000.00  41952.97  41996.49  54.22833   

                       return        rsi  func1buy

In [636]:
def printOut(func):
    for coin in coins:
        df = pd.read_sql(coin, engine).set_index('Time')
        print(coin)
        trades = func(df)['groups'].max() *2
        print(f"Qty: {trades}")
        costs = trades * 0.00075 # 0.075/100
        print(f"fee: {costs}")
        print(np.exp(func(df)['return'].sum())-1)
        print(np.exp(func(df)[['return','strategyreturn']].sum())-1-costs)

In [637]:
bb(test).to_string('BB')
smas(test).to_string('SMA')

#BB 2021-12-30 20:45:00  47428.43  47428.43  47351.36  47351.36

#SMA 2021-12-30 15:20:00  47489.67  47489.68  47475.61  47475.62    8.76160  1.407818e-03  10.765663  10.765507 



In [ ]:
printOut(smas)
#0.242057
#-0.045096

In [638]:
bb(test).to_string('testbb')
bb(test).position.diff().value_counts()

0.0    5740
Name: position, dtype: int64

In [639]:
##df['EMA200'] = ta.trend.ema_indicator(df.Close, window=200)
#df['wf_Top_bool'] = np.where(df['High'].rolling(9, center=True).max(), True, False)

#0.075/100

In [640]:
def comparemulti(df, *args):
    df = df.copy()
    buy = []
    for func in args:
        name = func.__name__
        print(name)
        strategy = func(df)
        #df[name + "BUY"] = strategy['buy']
        strategy['buy'].to_string('MAYBE')
        ##strategy.to_string('testxxx')
        #print(strategy['buy'])
        #strategy['buy']
        #df.to_string('johnnyyyyy')
        # print(val)

        # val = func(df)['buy']
        # df[val] = func(df)['buy']
        #arr = df["cluster"].to_numpy()
        strategy[name+'BUY!'] = strategy['buy']  
        df[name+"BUY"] = strategy['buy'] 

        ##df.to_string('dfx')
        # return         # print(val)return
        strategy.to_string('strat')   
    return strategy[[name+'buy']]



In [641]:
printOut(bb)

BTCUSDT
Qty: 368.0
fee: 0.276
-0.10902443530635086
return           -0.385024
strategyreturn   -0.383029
dtype: float64
ETHUSDT
Qty: 374.0
fee: 0.2805
-0.15621174413129457
return           -0.436712
strategyreturn   -0.434045
dtype: float64
BNBUSDT
Qty: 370.0
fee: 0.2775
-0.10628483567172309
return           -0.383785
strategyreturn   -0.383441
dtype: float64
SOLUSDT
Qty: 366.0
fee: 0.2745
-0.2133602073135632
return           -0.487860
strategyreturn   -0.486908
dtype: float64
ADAUSDT
Qty: 452.0
fee: 0.339
0.060785767234989185
return           -0.278214
strategyreturn   -0.277427
dtype: float64
XRPUSDT
Qty: 408.0
fee: 0.306
-0.11621524877699729
return           -0.422215
strategyreturn   -0.421582
dtype: float64
DOTUSDT
Qty: 408.0
fee: 0.306
-0.12018711766822765
return           -0.426187
strategyreturn   -0.424601
dtype: float64
LUNAUSDT
Qty: 442.0
fee: 0.3315
-0.07012655288517877
return           -0.401627
strategyreturn   -0.398050
dtype: float64
DOGEUSDT
Qty: 360.0
fee: 0.27
-0.040

In [642]:
stoch(test)['groups'].max()

229.0

In [643]:
#john = []  

#john = comparemulti(test, bb, rsistrat)
comparemulti(test, bb, rsistrat)
#print(john)
# 

bb
rsistrat


KeyError: "None of [Index(['rsistratbuy'], dtype='object')] are in the [columns]"